## Landsat Visualization 
Install the [Earth Engine Python API](https://developers.google.com/earth-engine/python_install) and [geehydro](https://github.com/giswqs/geehydro). The **geehydro** Python package builds on the [folium](https://github.com/python-visualization/folium) package and implements several methods for displaying Earth Engine data layers, such as `Map.addLayer()`, `Map.setCenter()`, `Map.centerObject()`, and `Map.setOptions()`. 
Uncomment these lines if you are running this notebook for the first time.

In [1]:
#!pip install earthengine-api
#!pip install geehydro

In [1]:
#Import libraries
import ee
import folium
import geehydro

Authenticate and initialize Earth Engine API. You only need to authenticate the Earth Engine API once. Uncomment the line `ee.Authenticate()` 
if you are running this notebook for the first time or if you are getting an authentication error.  

In [2]:
#ee.Authenticate()
ee.Initialize()

## Create an interactive map 
This step creates an interactive map using [folium](https://github.com/python-visualization/folium). The default basemap is the OpenStreetMap. Additional basemaps can be added using the `Map.setOptions()` function. 
The optional basemaps can be `ROADMAP`, `SATELLITE`, `HYBRID`, `TERRAIN`, or `ESRI`.


In [3]:
Map = folium.Map(location=[40, -100], zoom_start=4)
#Map.setOptions('HYBRID')
Map.setOptions('SATELLITE')

## Visualize Landsat Data  

In [4]:
#Load countries data
countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017");
Ethiopia = countries.filter(ee.Filter.eq("country_na","Ethiopia"));

landsat = ee.ImageCollection("LANDSAT/LC08/C01/T1") \
    .filterDate('2016-01-01', '2017-01-01') \
    .filterBounds(Ethiopia)

## this is a cloud masking method in EE
composite = ee.Algorithms.Landsat.simpleComposite(**{
  'collection': landsat,
  'asFloat': True
})

rgbVis = {'bands': ["B4", "B3", "B2"], 'min':0, 'max': 0.3}
nirVis = {'bands': ["B5", "B4", "B3"], 'min':0, 'max': [0.5, 0.3, 0.3]}

Map.addLayer(composite, rgbVis, "RGB")
Map.addLayer(composite, nirVis, "False Color")
Map.centerObject(Ethiopia, 12)

### Display 

In [5]:
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
Map